In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

llm = ChatOpenAI(model='gpt-4o-mini')

llm.invoke([HumanMessage(content="잘지냈어?")])

AIMessage(content='네, 잘 지내고 있습니다! 당신은 어떻게 지내고 계신가요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 12, 'total_tokens': 31, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_373a14eb6f', 'id': 'chatcmpl-DAWIthhc1VTHwVGl5XsDVNqbNfvF2', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c6fa8-4bff-7f01-a9e4-8e71eaae4c05-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 12, 'output_tokens': 19, 'total_tokens': 31, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [2]:
from langchain_core.tools import tool
from datetime import datetime
import pytz

@tool # @tool 데코레이터를 사용하여 함수를 도구로 등록 
def get_current_time(timezone: str, location: str) -> str:
    """ 현재 시각을 반환하는 함수
    
    Args:
        timezone (str): 타임존(예: Asia/Seoul). 실제 존재해야 함 
        location (str): 지역명. 타임존은 모든 지명에 대응하지 않으므로 이후 llm 답변 생성에 사용됨
    """

    tz = pytz.timezone(timezone)

    now = datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')

    location_and_local_time = f'{timezone} ({location}) 현재 시각 {now}' # 타임존, 지역명, 현재 시각을 문자열로 반환

    print(location_and_local_time)

    return location_and_local_time

In [4]:
# 도구를 tools 리스트에 추가하고, tool_dict에도 추가 
tools = [get_current_time,]
tool_dict = {"get_current_time": get_current_time,}

# 도구를 모델에 바인딩 : 모델에 도구를 바인딩하면, 도구를 사용하여 답변을 생성할 수 있음 
llm_with_tools = llm.bind_tools(tools)

In [5]:
from langchain_core.messages import SystemMessage

messages = [
    SystemMessage(content="너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
    HumanMessage(content="부산은 지금 몇시야?")
]

response = llm_with_tools.invoke(messages)
messages.append(response)

print(messages)

[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 129, 'total_tokens': 152, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_324beb61c9', 'id': 'chatcmpl-DAWnc777PbMAjLzYjqgzgnF4p16rG', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c6fc5-5dee-7241-8769-054a0723ddae-0', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': '부산'}, 'id': 'call_hp3RZ7hBRLIPvddOuoiLEZ2m', 'type': 'tool_call'}],

In [6]:
for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call['name']]
    print(tool_call['args'])
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

{'timezone': 'Asia/Seoul', 'location': '부산'}
Asia/Seoul (부산) 현재 시각 2026-02-18 17:06:22


[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 129, 'total_tokens': 152, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_324beb61c9', 'id': 'chatcmpl-DAWnc777PbMAjLzYjqgzgnF4p16rG', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c6fc5-5dee-7241-8769-054a0723ddae-0', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': '부산'}, 'id': 'call_hp3RZ7hBRLIPvddOuoiLEZ2m', 'type': 'tool_call'}

In [7]:
llm_with_tools.invoke(messages)

AIMessage(content='부산의 현재 시각은 2026년 2월 18일 17시 06분 22초입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 185, 'total_tokens': 214, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_324beb61c9', 'id': 'chatcmpl-DAWuY2Smyuh4fhEqDKepjx8oTMjW7', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c6fcb-ebca-7be1-9713-93157fe4716a-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 185, 'output_tokens': 29, 'total_tokens': 214, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})